In [5]:
# Skriptas užduoto ilgio sekų skaitymui iš MIT2ZIVE EKG įrašų

import pandas as pd
import numpy as np
from pathlib import Path
import sys

def read_seq_RR(rec_dir, all_beats_attr, idx, wl_side, wr_side):
# Nuskaito ir pateikia EKG seką apie R dantelį: wl_side - iš kairės pusės, wr_side - iš dešinės pusės,
# klasės numerį: 0, 1, 2.
# Taip pat pateikia RRl - EKG reikšmių skaičių nuo R dantelio iki prieš tai buvusio R dantelio,
# ir RRr - reikšmių skaičių nuo R dantelio iki sekančio R dantelio

    row = all_beats_attr.loc[idx]

    file_path = Path(rec_dir, str(row['userNr']) + '.npa')

    signal = np.load(file_path, mmap_mode='r')    
    signal_length = signal.shape[0]

    (seq_start, seq_end)  = get_seq_start_end(signal_length, row['sample'], wl_side, wr_side)

    RRl = row['RRl']
    RRr = row['RRr']

    # Ignoruojame per trumpas sekas įrašo pradžioje ir pabaigoje,
    # o taip pat pūpsnius, kuriems RRl ir RRr == -1
    if (seq_start == None or seq_end == None or RRl== -1 ):
        return None, None, None, None 
    else:    
        seq = signal[seq_start:seq_end]
        label = row['label']
    return seq, label, RRl, RRr


def get_seq_start_end(signal_length,i_sample,window_left_side,window_right_side):
    # Nustatome išskiriamos EKG sekos pradžią ir pabaigą
    seq_start = i_sample - window_left_side
    seq_end = i_sample + window_right_side
    if (seq_start < 0 or seq_end > signal_length):
        # print("\nseq_start: ", seq_start, " seq_end: ", seq_end)
        return (None,None)
    else:    
        return (seq_start, seq_end)


def anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=None, cols_pattern=None):
#  Skaičiuoja anotacijų pasiskirstymą
#  ind_lst - indeksų sąrašas, kuriuos reikia įtraukti į skaičiavimą

    if (ind_lst is not None):
        tmp_beats_attr = all_beats_attr.loc[all_beats_attr.index[ind_lst]]
        labels_table = pd.crosstab(index=tmp_beats_attr['userNr'], columns=tmp_beats_attr['symbol'], margins=True)
    else:
        labels_table = pd.crosstab(index=all_beats_attr['userNr'], columns=all_beats_attr['symbol'], margins=True)

    if (cols_pattern is not None):
        cols = list(labels_table.columns)
        cols_ordered = [s for s in cols_pattern if s in cols]
        labels_table = labels_table[cols_ordered]
    
    labels_sums = labels_table.sum(axis=1) 

    return labels_table, labels_sums
    

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'DUOM_VU'

# Nuoroda į DUOM_TST duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Anotacijoms priskirtos klasės
selected_beats = {'N':0, 'S':1, 'V':2}

print("\nBendras MIT ir Zive duomenų aplankas: ", Duomenu_aplankas)
print("MIT2ZIVE EKG įrašų aplankas: ", db_folder)


OS in my system :  win32

Bendras MIT ir Zive duomenų aplankas:  C:\DI\Data\MIT&ZIVE
MIT2ZIVE EKG įrašų aplankas:  DUOM_VU


In [6]:
import random
from scipy import stats
import datetime

# Įvairios operacijos, naudojant EKG įrašus ir all_beats_attr.csv

# Nuskaitome pūpsnių atributų failą
file_path = Path(db_path, 'all_beats_attr.csv')
all_beats_attr = pd.read_csv(file_path, index_col=0)

# I Nuskaito atsitiktinai parinktas sekas ir klasės nr
print("\nAtsitiktinių sekų skaitymas ir jų statistikos skaičiavimas\n")
wl_side, wr_side = 100, 100
randomlist = random.sample(range(0, len(all_beats_attr)), 3)

for idx in randomlist:
    seq_1d, label, RRl, RRr = read_seq_RR(db_path, all_beats_attr, idx, wl_side, wr_side)
    if (label != None):
        print("\n")
        print(f"{'idx ='} {idx}  {'seq_1d.shape ='} {seq_1d.shape}  {'label ='} {label}")
        print(f"mean = {np.mean(seq_1d):5.2f}, RRl = {RRl}  RRr = {RRr}")

# II Anotacijų pasiskirstymas visuose duomenyse
print("\nAnotacijų pasiskirstymas visuose duomenyse")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, cols_pattern=['N', 'S', 'V'])
print(labels_table)
# print("\n", labels_sums)


# III Suformuojame indeksų masyvus mokymo ir vertinimo imtims

# Train imties pacientai
DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 208, 209, 215, 220, 223,230]

# Validate imties pacientai
DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]


# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 1


# MOKYMO IMTIS

# Suformuojame mokymo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

# Visi pūpsnių indeksai
index_beats_attr = all_beats_attr.index

train_ind_lst = []
for userNr in DS1:
    selected_ind = index_beats_attr[all_beats_attr['userNr']==userNr]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    train_ind_lst.extend(selected_ind.to_list())

# Įrašome mokymo imties sekų indeksų sąrašą į diską
file_path = Path(db_path, 'train_ind_lst.csv')
np.savetxt(file_path, np.array(train_ind_lst), delimiter=',', fmt='%d')
print("\nMokymo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Anotacijų pasiskirstymas mokymo duomenyse
print("\nAnotacijų pasiskirstymas mokymo duomenyse\n")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=train_ind_lst, cols_pattern=['N', 'S', 'V'])
print(labels_table)
# print("\n", labels_sums)


# VERTINIMO IMTIS

# Suformuojame vertinimo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus
validate_ind_lst = []
for userNr in DS2:
    selected_ind = index_beats_attr[all_beats_attr['userNr']==userNr]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    validate_ind_lst.extend(selected_ind.to_list())

# Įrašome vertinimo imties sekų indeksų sąrašą į diską
file_path = Path(db_path, 'validate_ind_lst.csv')
np.savetxt(file_path, np.array(validate_ind_lst), delimiter=',', fmt='%d')
print("\nVertinimo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\nAnotacijų pasiskirstymas vertinimo duomenyse\n")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=validate_ind_lst, cols_pattern=['N', 'S', 'V'])
print(labels_table)
# print("\n", labels_sums)

print("\nPabaiga")




Atsitiktinių sekų skaitymas ir jų statistikos skaičiavimas



idx = 20483  seq_1d.shape = (200,)  label = 0
mean = -0.14, RRl = 213  RRr = 200


idx = 6484  seq_1d.shape = (200,)  label = 0
mean = -0.14, RRl = 139  RRr = 147


idx = 22728  seq_1d.shape = (200,)  label = 0
mean = -0.23, RRl = 193  RRr = 188

Anotacijų pasiskirstymas visuose duomenyse
symbol      N     S     V
userNr                   
100      2239    33     1
101      1860     3     0
103      2082     2     0
105      2526     0    41
106      1507     0   520
108      1740     4    17
109      2492     0    38
111      2123     0     1
112      2537     2     0
113      1789     6     0
114      1820    12    43
115      1953     0     0
116      2302     1   109
117      1534     1     0
118      2166    96    16
119      1543     0   444
121      1861     1     1
122      2476     0     0
123      1515     0     3
124      1536    31    47
200      1743    30   826
201      1635   128   198
202      2061    55    